# Correlation Analysis

This notebook analyzes the relationship between news sentiment and stock price movements.

**Analysis Steps:**
1. Merge sentiment data with stock prices
2. Calculate daily sentiment scores
3. Compute correlation between sentiment and price returns
4. Visualize the relationship

**Key Question:** Does positive news sentiment predict stock price increases?

## Setup and Load Data

In [ ]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime, timedelta

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Load sentiment data
company_news = pd.read_csv('../data/processed/company_news_with_sentiment.csv')
macro_news = pd.read_csv('../data/processed/macro_news_with_sentiment.csv')

# Convert dates
company_news['date'] = pd.to_datetime(company_news['date'])
macro_news['date'] = pd.to_datetime(macro_news['date'])

print("Data loaded:")
print(f"  Company news: {len(company_news)} articles")
print(f"  Macro news: {len(macro_news)} articles")
print(f"  Date range: {company_news['date'].min().date()} to {company_news['date'].max().date()}")

## Calculate Daily Sentiment Scores

Aggregating multiple articles per day into a single daily sentiment score for each stock.

In [ ]:
print("Calculating daily sentiment scores...\n")

# Company-specific daily sentiment (average sentiment per day per stock)
daily_sentiment = company_news.groupby(['ticker', 'date'])['sentiment_score'].agg(['mean', 'count']).reset_index()
daily_sentiment.columns = ['ticker', 'date', 'sentiment_score', 'article_count']

# Macro daily sentiment (average across all macro news per day)
macro_daily = macro_news.groupby('date')['sentiment_score'].mean().reset_index()
macro_daily.columns = ['date', 'macro_sentiment']

print("✓ Daily sentiment scores calculated")
print(f"\nSentiment statistics by stock:")
print(daily_sentiment.groupby('ticker')['sentiment_score'].describe())

## Fetch Stock Prices and Merge with Sentiment

Retrieving historical stock prices for the same time period as our news data.

In [ ]:
# Fetch stock prices
tickers = ['MSFT', 'TSLA', 'V']
start_date = company_news['date'].min()
end_date = company_news['date'].max()

print(f"Fetching stock prices from {start_date.date()} to {end_date.date()}...\n")

all_stock_data = []

for ticker in tickers:
    stock = yf.Ticker(ticker)
    df = stock.history(start=start_date, end=end_date + timedelta(days=1))
    df['ticker'] = ticker
    df['date'] = pd.to_datetime(df.index.date)
    
    # Calculate daily returns (% change from previous day)
    df['daily_return'] = df['Close'].pct_change() * 100
    
    all_stock_data.append(df[['date', 'ticker', 'Close', 'daily_return']])
    print(f"{ticker}: {len(df)} trading days")

# Combine all stock data
stock_prices = pd.concat(all_stock_data, ignore_index=True)
print(f"\n✓ Stock data collected: {len(stock_prices)} total records")

In [ ]:
# Merge stock prices with sentiment
merged_data = stock_prices.merge(daily_sentiment, on=['ticker', 'date'], how='left')
merged_data = merged_data.merge(macro_daily, on='date', how='left')

# Fill missing sentiment with 0 (days with no news)
merged_data['sentiment_score'] = merged_data['sentiment_score'].fillna(0)
merged_data['macro_sentiment'] = merged_data['macro_sentiment'].fillna(0)
merged_data['article_count'] = merged_data['article_count'].fillna(0)

print(f"✓ Merged dataset created: {len(merged_data)} records")
print("\nSample of merged data:")
merged_data.head(10)

## Data Coverage Analysis

Understanding how many days have news coverage vs. no news.

In [ ]:
print("Data Coverage Analysis:\n")

for ticker in ['MSFT', 'TSLA', 'V']:
    ticker_data = merged_data[merged_data['ticker'] == ticker]
    
    total_days = len(ticker_data)
    days_with_news = len(ticker_data[ticker_data['article_count'] > 0])
    days_without_news = total_days - days_with_news
    
    print(f"{ticker}:")
    print(f"  Total trading days: {total_days}")
    print(f"  Days WITH news: {days_with_news} ({days_with_news/total_days*100:.1f}%)")
    print(f"  Days WITHOUT news: {days_without_news} ({days_without_news/total_days*100:.1f}%)")
    if days_with_news > 0:
        print(f"  Avg articles per day (when news exists): {ticker_data[ticker_data['article_count'] > 0]['article_count'].mean():.1f}")
    print()

# Macro news coverage
days_with_macro = len(merged_data[merged_data['macro_sentiment'] != 0]['date'].unique())
total_unique_days = len(merged_data['date'].unique())

print(f"Macro news coverage:")
print(f"  Days with macro news: {days_with_macro}/{total_unique_days} ({days_with_macro/total_unique_days*100:.1f}%)")

## Correlation Analysis

Testing whether news sentiment correlates with stock price movements.

We examine two relationships:
- **Same-day correlation:** Does sentiment correlate with that day's price movement?
- **Next-day correlation:** Does sentiment predict the next day's price movement?

In [ ]:
print("="*80)
print("SENTIMENT vs STOCK PRICE CORRELATION")
print("="*80)

correlations = []

for ticker in ['MSFT', 'TSLA', 'V']:
    ticker_data = merged_data[merged_data['ticker'] == ticker].copy()
    
    # Only analyze days with news
    with_news = ticker_data[ticker_data['article_count'] > 0]
    
    if len(with_news) > 5:  # Need at least 5 data points
        # Calculate next-day returns
        with_news['next_day_return'] = with_news['daily_return'].shift(-1)
        
        # Correlation coefficients
        corr_same_day = with_news['sentiment_score'].corr(with_news['daily_return'])
        corr_next_day = with_news['sentiment_score'].corr(with_news['next_day_return'])
        
        print(f"\n{ticker}:")
        print(f"  Days with news: {len(with_news)}")
        print(f"  Same-Day Correlation: {corr_same_day:.3f}")
        print(f"  Next-Day Correlation: {corr_next_day:.3f}")
        
        correlations.append({
            'ticker': ticker,
            'same_day': corr_same_day,
            'next_day': corr_next_day,
            'sample_size': len(with_news)
        })

corr_df = pd.DataFrame(correlations)

print("\n" + "="*80)
print("SUMMARY")
print("="*80)
print(corr_df.to_string(index=False))

print("\n💡 Interpretation:")
print("  Positive (>0.3): Good news tends to correlate with price increases")
print("  Negative (<-0.3): Contrarian indicator (good news → price decreases)")
print("  Near zero (-0.3 to 0.3): No clear relationship")
print("\n⚠️ Note: Small sample sizes limit statistical significance")

## Visualization

Scatter plots showing the relationship between sentiment scores and next-day stock returns.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, ticker in enumerate(['MSFT', 'TSLA', 'V']):
    ticker_data = merged_data[merged_data['ticker'] == ticker].copy()
    with_news = ticker_data[ticker_data['article_count'] > 0].copy()
    with_news['next_day_return'] = with_news['daily_return'].shift(-1)
    
    # Scatter plot
    axes[idx].scatter(with_news['sentiment_score'], with_news['next_day_return'], 
                     alpha=0.6, s=100, color='steelblue')
    
    # Reference lines
    axes[idx].axhline(y=0, color='gray', linestyle='--', alpha=0.3, linewidth=1)
    axes[idx].axvline(x=0, color='gray', linestyle='--', alpha=0.3, linewidth=1)
    
    # Labels and title
    axes[idx].set_xlabel('News Sentiment Score', fontsize=12)
    axes[idx].set_ylabel('Next-Day Return (%)', fontsize=12)
    
    corr_value = corr_df[corr_df['ticker']==ticker]['next_day'].values[0]
    axes[idx].set_title(f'{ticker}\nCorrelation: {corr_value:.2f}', 
                       fontsize=14, fontweight='bold')
    axes[idx].grid(True, alpha=0.3)

plt.suptitle('News Sentiment vs Next-Day Stock Returns', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('../data/outputs/sentiment_correlation.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Visualization saved to data/outputs/sentiment_correlation.png")

## Summary

**Key Findings:**
- Weak correlations across all three stocks (r < 0.6)
- VISA shows the strongest next-day correlation (+0.52)
- Small sample sizes (12-18 days with news per stock) limit statistical significance

**Limitations:**
- 30-day time window too short for robust conclusions
- Daily closing prices miss intraday reactions to news
- Equal weighting of all articles (minor vs. major news)

**Next Steps:**
See `04_summary.ipynb` for complete project conclusions and recommendations.